In [4]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:80% !important; }</style>"))

/tmp/ipykernel_578573/1960957901.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [5]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic
%config Completer.use_jedi = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [6]:
import pyphi
from visualize_pyphi import *
from visualize_pyphi import utils
import pickle as pkl
from tqdm.auto import tqdm
import numpy as np
import itertools
from pyphi.models.subsystem import FlatCauseEffectStructure as sep
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

/tmp/ipykernel_578573/439437236.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [7]:
'''
Check that the config parameters are correct (if not change pyphi_config.yml to match):
REPERTOIRE_DISTANCE = 'maximal-state-first'
IIT_VERSION = 'IIT_4.0_SMALL_PHI'
PARTITION_TYPE = 'TRI'
'''
pyphi.config.REPERTOIRE_DISTANCE,pyphi.config.IIT_VERSION,pyphi.config.PARTITION_TYPE

('IIT_4.0_SMALL_PHI', 'maximal-state-first', 'TRI')

In [8]:
# net_name = "space_7"

# th = 1 / 4
# exp = 5

# s = .8
# l = 0.26
# f = 0.06
# g = 0.0

# weights = np.array(
#     [
#         [s, l, f, g, 0, 0, 0],  # A
#         [l, s, l, f, g, 0, 0],  # B
#         [f, l, s, l, f, g, 0],  # C
#         [g, f, l, s, l, f, g],  # D
#         [0, g, f, l, s, l, f],  # E
#         [0, 0, g, f, l, s, l],  # F
#         [0, 0, 0, g, f, l, s],  # G        
#         #A, B, C, D, E, F, G
#     ]
# )

# # weights += np.random.random((6,6))/100
# # weights = np.abs(weights)

# node_labels = ["A", "B", "C", "D", "E", "F", "G"]
# mech_func = [f for f in "sssssss"]

# network = ng.get_net(
#     mech_func,
#     weights,
#     exp=exp,
#     th=th,
#     node_labels=node_labels,
#     network_name=net_name,
#     pickle_network=True,
# )

# state = (0, 0, 0, 0, 0, 0, 0)


# subsystem = pyphi.Subsystem(network, state)

# utils.pklthis(subsystem,'example_data/7_node_grid_subsystem.pkl')

In [9]:

C = pyphi.direction.Direction.CAUSE
E = pyphi.direction.Direction.EFFECT


In [10]:
def make_bag(all_mices):
    bag = dict()
    for mice in all_mices:
        m = mice.mechanism
        d = mice.direction
        p = mice.purview

        if m in bag.keys():
            if d in bag[m].keys():
                bag[m][d][p] = mice
            else:
                bag[m][d] = dict()
                bag[m][d][p] = mice
        else:
            bag[m] = dict()
            bag[m][d] = dict()
            bag[m][d][p] = mice

    return bag

def get_bag_of_mices(subsystem, mechanisms, purviews, candidate="irreducible"):

    if candidate == "irreducible":
        return {
            mechanism: {
                direction: {
                    purview: subsystem.find_mice(direction, mechanism, (purview,))
                    for purview in purviews
                }
                for direction in [
                    C,
                    E,
                ]
            }
            for mechanism in mechanisms
        }

    elif candidate == "miw":

        # compute candidate mices for every purview
        all_mices = []

        for mechanism in tqdm(mechanisms):
            for direction in [C, E]:
                candidate_mices = [
                    subsystem.find_mice(direction, mechanism, (purview,))
                    for purview in purviews
                ]
                # check if each mice is maximally irreducible within
                candidate_mices_MIW = [
                    mice
                    for mice in candidate_mices
                    if mice.phi > 0
                    and not any(
                        [
                            all([unit in mice.purview for unit in mice2.purview])
                            and mice2.phi > mice.phi
                            for mice2 in candidate_mices
                        ]
                    )
                ]

                all_mices.extend(candidate_mices_MIW)

        bag = make_bag(all_mices)
        return bag

In [11]:
def candidate_concepts(bag_of_mices, method='maximize_phi'):

    if method=='maximize_phi':
    # Pick the maximal mic and mie for every mechanism represented in the bag of mices
        concepts = dict()
        for mechanism, all_mices in bag_of_mices.items():
            max_mices = []
            for direction, directed_mices in all_mices.items():
                phi = 0
                max_mice = None
                for purview, mice in directed_mices.items():

                    if mice.phi >= phi:
                        max_mice = mice
                        phi = mice.phi

                if phi > 0:
                    max_mices.append((max_mice, phi))

            if len(max_mices) > 1:
                concepts[(max_mices[0][0], max_mices[1][0])] = min(
                    [max_mices[0][1], max_mices[1][1]]
                )

        if len(concepts) < 1:
            return None
        else:
            return concepts
    
#     if method=='reflexivity':
#     # Pick the purviews over the mechanism itself first 

In [12]:
def prune_bag_of_mices(bag_of_mices, concept):

    new_bag = {
        mechanism: {
            direction: {
                purview: bag_of_mices[mechanism][direction][purview]
                for purview, mice in directed_mices.items()
                if (
                    not mechanism == concept[0].mechanism
                    and not (
                        concept[0].direction == direction
                        and concept[0].purview == purview
                    )
                )
                and (
                    not mechanism == concept[1].mechanism
                    and not (
                        concept[1].direction == direction
                        and concept[1].purview == purview
                    )
                )
            }
            for direction, directed_mices in all_mices.items()
        }
        for mechanism, all_mices in bag_of_mices.items()
    }
    return new_bag

In [13]:
def CES_from_bag_of_mices(bag_of_mices):
    max_concepts = []
    while len(bag_of_mices):
        concepts = candidate_concepts(bag_of_mices)
        if concepts == None:
            break

        max_concept = sorted(concepts, reverse=True)[0]
        bag_of_mices = prune_bag_of_mices(bag_of_mices, max_concept)

        max_concepts.extend([mice for mice in max_concept])

    return sep(max_concepts)

# Space

In [14]:
with open(r"example_data/4_node_grid_subsystem.pkl", "rb") as f:
    subsystem = pkl.load(f)

In [15]:
ces = pyphi.compute.ces(subsystem)

In [16]:
relations = list(pyphi.relations.relations(subsystem,sep(ces),max_degree=2))

  0%|          | 0/334 [00:00<?, ?it/s]

/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON


In [17]:
sia = pyphi.big_phi.sia(subsystem,ces,relations)

2022-01-30 13:26:25,592	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8266


Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/2 [00:00<?, ?it/s]

Evaluating maximal compositional state


In [18]:
sia.phi

0.014032332339088526

In [19]:
mechanisms = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
purviews = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
bag_of_mices = get_bag_of_mices(subsystem,mechanisms,purviews,candidate='miw')

  0%|          | 0/15 [00:00<?, ?it/s]

In [20]:
flat_bag = []
for mechanism in tqdm(mechanisms):
    for direction in [C,E]:
        for purview in mechanisms: 
            try:
                flat_bag.append(bag_of_mices[mechanism][direction][purview])
            except:
                pass

  0%|          | 0/15 [00:00<?, ?it/s]

In [21]:
reflexive_ds = [
    pyphi.subsystem.Concept(m1.mechanism, m1, m2, subsystem)
    for m1,m2 in itertools.combinations(flat_bag,2)
    if m1.mechanism==m2.mechanism==m1.purview==m2.purview
    and m1.direction == C
    and m2.direction == E
]

In [22]:
reflexive_sep = sep(pyphi.models.CauseEffectStructure(reflexive_ds))

In [23]:
reflexive_causes = [m.purview for m in reflexive_sep if m.direction==C]
reflexive_effects = [m.purview for m in reflexive_sep if m.direction==E]
reflexive_mechanisms = [m.mechanism for m in reflexive_sep]

remaining_mices = []
for m in flat_bag:
    if m.direction==C and m.mechanism not in reflexive_mechanisms and m.purview not in reflexive_causes:
        remaining_mices.append(m)
    if m.direction==E and m.mechanism not in reflexive_mechanisms and m.purview not in reflexive_effects:
        remaining_mices.append(m)

In [24]:
remaining_ces = CES_from_bag_of_mices(make_bag(remaining_mices))
utils.sepces2df(remaining_ces,subsystem)

,mechanism,direction,purview,state,phi
0,ABD,CAUSE,ABD,000,0.018262
1,ABD,EFFECT,AD,00,0.003075


In [25]:
final_ces = compute.get_linked_ces(list(reflexive_sep)+list(remaining_ces),subsystem)
final_ces.subsystem=subsystem

In [26]:
relations = list(pyphi.relations.relations(subsystem,sep(final_ces),max_degree=2,parallel=True))

  0%|          | 0/237 [00:00<?, ?it/s]

/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/

In [27]:
sia = pyphi.big_phi.sia(subsystem,final_ces,relations)

Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/1 [00:00<?, ?it/s]

Evaluating maximal compositional state


In [28]:
sia.phi

0.04542253372529282

In [29]:
utils.sepces2df(sep(sia.phi_structure.distinctions),subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.520382
1,A,EFFECT,A,0,0.656129
2,B,CAUSE,B,0,0.629285
3,B,EFFECT,B,0,0.713724
4,C,CAUSE,C,0,0.629285
5,C,EFFECT,C,0,0.713724
6,D,CAUSE,D,0,0.520382
7,D,EFFECT,D,0,0.656129
8,AB,CAUSE,AB,00,0.071485
9,AB,EFFECT,AB,00,0.069119


# Ch 3

In [53]:
th = 1/4
exp = 5
mu = 1
si = .3
ll = 1
kk = 15
x0 = .7

q = .4
s = .8
l = .25
j = 1-q
i = j-.05
b = 0.01

weights = np.array([
    [s,l,0,i,0,0],#A
    [l,s,l,j,b,0],#B
    [0,l,s,i,0,0],#C
    [b,b,b,q,0,i],#D
    [0,i,0,0,0,0],#I
    [0,0,0,0,0,0],#O
    #A,B,C,D,I,O
  ])

node_labels = ['A','B','C','D','I','O']
mech_func = [f for f in 'sssgll']
network = network_generator.get_net(mech_func, weights, exp=exp, th=th, mu=mu, si=si, l=ll, k=kk, x0=x0, node_labels=node_labels, pickle_network=False)
# utils.pklthis(network,'example_data/ch3_network.pkl')

  0%|          | 0/64 [00:00<?, ?it/s]

In [54]:
# network = utils.loadpkl('example_data/ch3_network.pkl')
subsystem = pyphi.Subsystem(network,state=(0,1,0,1,1,1),nodes=(0,1,2,3))

In [55]:
ces = pyphi.compute.ces(subsystem)
# sepces = sep(ces)
# utils.sepces2df(sepces,subsystem)

In [56]:
sepces = sep(ces)

In [57]:
relations = list(pyphi.relations.relations(subsystem,sepces,max_degree=5))

  0%|          | 0/10793 [00:00<?, ?it/s]

In [52]:
sia = pyphi.big_phi.sia(subsystem,ces,relations)

Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/2 [00:00<?, ?it/s]

RayTaskError(TypeError): [36mray::_max_system_intrinsic_information()[39m (pid=588722, ip=144.92.2.85)
  File "/home/mgrasso/pyphi/pyphi/big_phi.py", line 600, in _max_system_intrinsic_information
    return max(
TypeError: '>' not supported between instances of 'PhiStructure' and 'PhiStructure'

2022-01-30 13:39:58,553	ERROR worker.py:84 -- Unhandled error (suppress with RAY_IGNORE_UNHANDLED_ERRORS=1): ray::_max_system_intrinsic_information() (pid=588727, ip=144.92.2.85)
  File "/home/mgrasso/pyphi/pyphi/big_phi.py", line 600, in _max_system_intrinsic_information
    return max(
TypeError: '>' not supported between instances of 'PhiStructure' and 'PhiStructure'


In [ ]:
sia.phi

In [ ]:
utils.sepces2df(sep(sia.phi_structure.distinctions),subsystem)

2022-01-28 14:52:49,136	ERROR worker.py:84 -- Unhandled error (suppress with RAY_IGNORE_UNHANDLED_ERRORS=1): ray::_max_system_intrinsic_information() (pid=1909983, ip=144.92.2.85)
  File "/home/mgrasso/pyphi/pyphi/big_phi.py", line 595, in _max_system_intrinsic_information
    return max(
TypeError: '>' not supported between instances of 'PhiStructure' and 'PhiStructure'


In [44]:
mechanisms = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
purviews = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
bag_of_mices = get_bag_of_mices(subsystem,mechanisms,purviews,candidate='miw')

  0%|          | 0/15 [00:00<?, ?it/s]

In [45]:
flat_bag = []
for mechanism in tqdm(mechanisms):
    for direction in [C,E]:
        for purview in mechanisms: 
            try:
                flat_bag.append(bag_of_mices[mechanism][direction][purview])
            except:
                pass

  0%|          | 0/15 [00:00<?, ?it/s]

In [46]:
reflexive_ds = [
    pyphi.subsystem.Concept(m1.mechanism, m1, m2, subsystem)
    for m1,m2 in itertools.combinations(flat_bag,2)
    if m1.mechanism==m2.mechanism==m1.purview==m2.purview
    and m1.direction == C
    and m2.direction == E
]

In [47]:
reflexive_sep = sep(pyphi.models.CauseEffectStructure(reflexive_ds))

In [48]:
reflexive_causes = [m.purview for m in reflexive_sep if m.direction==C]
reflexive_effects = [m.purview for m in reflexive_sep if m.direction==E]
reflexive_mechanisms = [m.mechanism for m in reflexive_sep]

remaining_mices = []
for m in flat_bag:
    if m.direction==C and m.mechanism not in reflexive_mechanisms and m.purview not in reflexive_causes:
        remaining_mices.append(m)
    if m.direction==E and m.mechanism not in reflexive_mechanisms and m.purview not in reflexive_effects:
        remaining_mices.append(m)

In [49]:
remaining_ces = CES_from_bag_of_mices(make_bag(remaining_mices))
utils.sepces2df(remaining_ces,subsystem)

,mechanism,direction,purview,state,phi
0,AD,CAUSE,AB,01,0.146204
1,AD,EFFECT,AB,00,0.005580
2,BD,CAUSE,BD,10,0.092453
3,BD,EFFECT,AC,00,0.003000
4,CD,CAUSE,BC,10,0.146204
5,CD,EFFECT,BC,00,0.005580
6,ABC,CAUSE,AC,00,0.070977
7,ABC,EFFECT,ABC,010,0.023489
8,ACD,CAUSE,ABCD,0101,0.209839
9,ACD,EFFECT,BD,01,0.061841


In [50]:
final_ces = compute.get_linked_ces(list(reflexive_sep)+list(remaining_ces),subsystem)
final_ces.subsystem=subsystem

In [52]:
relations = list(pyphi.relations.relations(subsystem,sep(final_ces),max_degree=2,parallel=True))

  0%|          | 0/91 [00:00<?, ?it/s]

/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/

In [53]:
sia = pyphi.big_phi.sia(subsystem,final_ces,relations)

Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/1 [00:00<?, ?it/s]

Evaluating maximal compositional state


In [54]:
sia.phi

0.00011935240266181778

In [55]:
utils.sepces2df(sep(sia.phi_structure.distinctions),subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.486034
1,A,EFFECT,A,0,0.627340
2,B,CAUSE,B,1,0.232803
3,B,EFFECT,B,1,0.352352
4,C,CAUSE,C,0,0.486034
5,C,EFFECT,C,0,0.627340
6,D,CAUSE,D,0,0.010567
7,D,EFFECT,D,0,0.009994
8,AD,CAUSE,AB,01,0.146204
9,AD,EFFECT,AB,00,0.005580
